In [2]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from group_recommender_system import *
import networkx as nx
import random

G = nx.read_gpickle("C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\vegas_graph_small.gpickle")

In [4]:
import statistics

def median_category_rating(G, category):
    ratings = []
    for rest in G.neighbors(category):
        for user in G.neighbors(rest):
            if G.nodes[user]['type'] == 'user':
                ratings.append(G[rest][user]['rating'])
    return statistics.mean(ratings)

In [5]:
def item_biases_by_category(G, category, median_rating):
    biases = {}
    for rest in G.neighbors(category):
        ratings = []
        for user in G.neighbors(rest):
            if G.nodes[user]['type'] == 'user':
                ratings.append(G[rest][user]['rating'])
        biases[rest] = statistics.mean(ratings) - median_rating
    return biases

In [6]:
def user_biases_by_category(G, category, median_rating):
    users = set()
    for rest in G.neighbors(category):
        for user in G.neighbors(rest):
            if G.nodes[user]['type'] == 'user':
                users.add(user)
    
    biases = {}
    for user in users:
        ratings = []
        for restaurant in G.neighbors(user):
            if G.has_edge(restaurant, category):
                ratings.append(G[user][restaurant]['rating'])
        biases[user] = statistics.mean(ratings) - median_rating
    return biases

In [7]:
def user_item_biases(G, user_biases, item_biases, median_rating):
    biases = {}
    for u in user_biases:
        biases[u] = {}
        for i in item_biases:
            biases[u][i] = median_rating + user_biases[u] + item_biases[i]
    return biases

In [16]:
#test_user = random.choice(get_nodes_of_type(G, 'user'))
test_user = 'BIO98HB3ZDpNpQT49q3djQ'
print("User: ", test_user, "\n")
preferences = category_preference_list(G, test_user)
print("Preferences:", preferences, "\n")

category = 'Wraps'
print("***" , category, "***\n")

median_rating = median_category_rating(G, category) 
print("Median: ", median_rating)

user_biases = user_biases_by_category(G, category, median_rating)
print("User Biases: ", len(user_biases))

item_biases = item_biases_by_category(G, category, median_rating)
print("Item Biases: ", len(item_biases.keys()))

u_i_biases = user_item_biases(G, user_biases, item_biases, median_rating)

for user in user_biases:
    for item in item_biases:
        if G.has_edge(user, item):
            print(G.nodes[user]['name']," to ", G.nodes[item]['name'], " --- ", G[user][item]['rating'], "-", u_i_biases[user][item])
print()

print("Rating predictions")
for r in G.neighbors(category):
    if(r not in G.neighbors(test_user)):
        print(" ---", G.nodes[r]['name'], "-", u_i_biases[test_user][r])
       

User:  BIO98HB3ZDpNpQT49q3djQ 

Preferences: {'Pubs': 4, 'Bars': 4, 'American (Traditional)': 4, 'Nightlife': 4, 'Food Delivery Services': 4, 'Salad': 4, 'Wraps': 4, 'Caterers': 4, 'Food': 4, 'Sandwiches': 4, 'American (New)': 4, 'Event Planning & Services': 4, 'Soup': 4} 

*** Wraps ***

Median:  3.8208955223880596
User Biases:  67
Item Biases:  15
Shannon  to  Earl of Sandwich  ---  3 - 3.3742264288314527
Zulett  to  Angelina's Pizza  ---  2 - -0.32089552238805963
Karen  to  Earl of Sandwich  ---  4 - 4.374226428831452
Christian  to  Earl of Sandwich  ---  5 - 5.374226428831452
Wild  to  Tropical Smoothie Cafe  ---  1 - -0.32089552238805963
Arturo  to  Earl of Sandwich  ---  5 - 5.374226428831452
Ariana  to  Earl of Sandwich  ---  5 - 5.374226428831452
T  to  Greens & Grill  ---  5 - 4.845771144278607
Christel  to  Earl of Sandwich  ---  5 - 5.374226428831452
Julie  to  Earl of Sandwich  ---  3 - 3.3742264288314527
Brock  to  Earl of Sandwich  ---  5 - 5.374226428831452
Kevin  to  Ea

In [23]:
user = random.choice(get_nodes_of_type(G, 'user'))
similar_users = {}
for r in G.neighbors(user):
        for c in G.neighbors(r): 
            if G.nodes[c]['type'] == 'user' and c != user and c not in similar_users:
                similar_users[c] = category_preference_list(G, c)
print(similar_users)

{'4A92tEwfs7tq4APkSt2ytA': {'Meat Shops': 5, 'Butcher': 5, 'Food': 5, 'Steakhouses': 5, 'Specialty Food': 5}, 'BXRlUjTE5A6ktc5pZbU3TA': {'Meat Shops': 5, 'Butcher': 5, 'Food': 5, 'Steakhouses': 5, 'Specialty Food': 5}, '5005LUoMw8RJckJT_w9u3A': {'Meat Shops': 2, 'Butcher': 2, 'Food': 2, 'Steakhouses': 2, 'Specialty Food': 2}, 'KVwTekIxO50OCC90t8Pp5A': {'Meat Shops': 5, 'Butcher': 5, 'Food': 5, 'Steakhouses': 5, 'Specialty Food': 5}, 'HzQEnTmOYsgVkTQvQCw4Jg': {'Meat Shops': 5, 'Butcher': 5, 'Food': 5, 'Steakhouses': 5, 'Specialty Food': 5}, 'nBTk93i1C__uQ8_-Vdj5_w': {'Meat Shops': 3, 'Butcher': 3, 'Food': 3, 'Steakhouses': 3, 'Specialty Food': 3}, 'PnaEwJPxwuYVIO269NZE0A': {'Meat Shops': 5, 'Butcher': 5, 'Food': 5, 'Steakhouses': 5, 'Specialty Food': 5}, 'a6tBYL5nqGe6CMXSjN_eDQ': {'Meat Shops': 3, 'Butcher': 3, 'Food': 3, 'Steakhouses': 3, 'Specialty Food': 3}, 'E0NyDRY26hv5XEsyxaI2YQ': {'Meat Shops': 4, 'Butcher': 4, 'Food': 4, 'Steakhouses': 4, 'Specialty Food': 4}, 'bivRZ7s6Z6cZsu1u_

In [12]:
#test_user = 'YrSb656bOiCRFlAeNYqQ-Q'
#test_user = 'hGYGNZsPhcRuLgUVWR_rjQ'
test_user = '_Ojr37qJFwX0zZ2S0WtVqQ'
#test_user = random.choice(get_nodes_of_type(G, 'user'))
print("User: ", test_user)

other_users = most_similar_users(G, test_user)
print("Similar users: ", other_users)

recommendations = set()
for u in other_users:
    recommendations.update(recommend_restaurants(G, u, test_user))

nbrs = G.neighbors(test_user)

for n in nbrs:
    for r in recommendations:
        print("*", G.nodes[n]['name'], "-", G.nodes[r]['name'], " --- ", restaurant_similarity(G, n, r))


User:  _Ojr37qJFwX0zZ2S0WtVqQ
Similar users:  ['s_JDz1gBlk8nDgwiFj9_cA', 'XMv3rplCwRGTuzYA2VwzRQ', 'pirzWkMyq6SahyufAe8jVg', 'F6lHT23hTEjOg8FRy9Iv3Q', 'j2Q3ObT4qmBuI7UpLSu0Ig', 'FOy0Qrkw54VKDEWIw7T-jw', 'UNV0c172_2RZBkviuwFQfQ']
* Wicked Spoon - Pt's Pub  ---  0.1111111111111111


In [17]:
#test_user = 'YrSb656bOiCRFlAeNYqQ-Q'
#test_user = 'hGYGNZsPhcRuLgUVWR_rjQ'
#test_user = '_Ojr37qJFwX0zZ2S0WtVqQ' #este tem varias categorias
test_user = random.choice(get_nodes_of_type(G, 'user'))
print("User: ", test_user, "\n")

similar_users = most_similar_users(G, test_user)
print("Similar users: ", similar_users)

category_ratings = predict_category_ratings(G, test_user, similar_users)
print(category_ratings, "\n")

recommendations = set()
for u in similar_users:
    recommendations.update(recommend_restaurants(G, u, test_user))

for rest in recommendations:
    ratinglist = {}
    for cat in G.neighbors(rest):
        if G.nodes[cat]['type'] == 'category':
            ratinglist[cat] = category_ratings[cat]
            
    rating = predict_restaurant_ratings(G, category_ratings, rest)
    print("*", G.nodes[rest]['name'], "-", rating)
    print(" ", ratinglist, "\n")

User:  ld9Q7kd06mbl1kMzISS02g 

Similar users:  ['N3oNEwh0qgPqPP3Em6wJXw']
{'Ethiopian': 4, 'Vegetarian': 4, 'Cafes': 4, 'Breakfast & Brunch': 3.5, 'Bagels': 4, 'Juice Bars & Smoothies': 4, 'Food': 3.5, 'Coffee & Tea': 4, 'Bakeries': 4, 'Tapas/Small Plates': 3, 'Tapas Bars': 3, 'Nightlife': 3, 'American (New)': 3, 'Beer': 3, 'Wine & Spirits': 3, 'Bars': 3, 'Wine Bars': 3, 'Lounges': 3, 'British': 3, 'Pubs': 3, 'Southern': 5, 'American (Traditional)': 5, 'Soul Food': 5} 

* Madhouse Coffee - 3.8333333333333335
  {'Breakfast & Brunch': 3.5, 'Bagels': 4, 'Juice Bars & Smoothies': 4, 'Food': 3.5, 'Coffee & Tea': 4, 'Bakeries': 4} 

* Hearthstone Kitchen & Cellar - 3.090909090909091
  {'Tapas/Small Plates': 3, 'Food': 3.5, 'Tapas Bars': 3, 'Nightlife': 3, 'American (New)': 3, 'Beer': 3, 'Wine & Spirits': 3, 'Bars': 3, 'Wine Bars': 3, 'Lounges': 3, 'Breakfast & Brunch': 3.5} 

* Queen Victoria Pub - 3
  {'British': 3, 'Nightlife': 3, 'Pubs': 3, 'Bars': 3} 

* M&M Soul Food Cafe - 5
  {'South